# Augmentation File 

### This file uses the albumentations library to augment the images. The augmentations were only applied to day 0 to day 3 to strike a balance between these classes and abv_day_3 (days 4-7). The augmented images were saved with an “augmented” name extension to differentiate the original phots and augmented images. Only flip, and rotational transformations were applied to the images so as not to alter the resolution, dimensions and colours. After augmentations, there were a total of 621 images. The augmentations contributed to about 300 images. These augmentations were done over colab.

In [ ]:
from google.colab import drive
import os
import cv2
import albumentations as A
from xml.etree import ElementTree as ET
import shutil

drive.mount('/content/drive', force_remount=True)

def augment_and_save(image_folder, label_folder, output_folder):
    augmentation = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))

    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(os.path.join(output_folder, "images"), exist_ok=True)  # Create "images" directory

    image_files = os.listdir(image_folder)

    augmented_images = []
    augmented_labels = []

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)

        label_file = os.path.splitext(image_file)[0] + ".xml"
        label_path = os.path.join(label_folder, label_file)
        tree = ET.parse(label_path)
        root = tree.getroot()

        original_bboxes = []
        for obj in root.findall(".//object"):
            bbox = obj.find("bndbox")
            x_min = float(bbox.find("xmin").text)
            y_min = float(bbox.find("ymin").text)
            x_max = float(bbox.find("xmax").text)
            y_max = float(bbox.find("ymax").text)
            original_bboxes.append([x_min, y_min, x_max, y_max])

        augmented = augmentation(image=image, bboxes=original_bboxes, category_ids=[1] * len(original_bboxes))
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']

        transformed = False

        for original_bbox, augmented_bbox in zip(original_bboxes, augmented_bboxes):
            if any(orig != aug for orig, aug in zip(original_bbox, augmented_bbox)):
                transformed = True
                break

        if transformed:
            # Save augmented image
            output_image_file = os.path.splitext(image_file)[0] + "_augmented.jpg"
            output_image_path = os.path.join(output_folder, "images", output_image_file)
            cv2.imwrite(output_image_path, augmented_image)
            augmented_images.append(output_image_path)

            # Save augmented label (XML format)
            root.find("filename").text = output_image_file
            output_label_file = os.path.splitext(label_file)[0] + "_augmented.xml"
            output_label_path = os.path.join(output_folder, "labels", output_label_file)
            os.makedirs(os.path.dirname(output_label_path), exist_ok=True)
            with open(output_label_path, 'wb') as output_label_file:
                tree.write(output_label_file, encoding='utf-8')
            augmented_labels.append(output_label_path)

    return augmented_images, augmented_labels

# Set your main directory in Google Drive
main_directory = "/content/drive/MyDrive/capstone/Augmented Data/pascal"

# Get list of subclass directories
subclass_directories = [d for d in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, d))]

for subclass in subclass_directories:
    subclass_directory = os.path.join(main_directory, subclass)
    image_folder = os.path.join(subclass_directory, "images")
    label_folder = os.path.join(subclass_directory, "labels")
    output_folder = os.path.join(subclass_directory, "augment")

    # Save augmented images in output folder
    augmented_and_labels = augment_and_save(image_folder, label_folder, output_folder)

    # Create folder for merged data
    merged_folder = os.path.join(subclass_directory, 'merged_data')
    os.makedirs(merged_folder, exist_ok=True)

    # Copying original images into merged folder
    original_images = os.path.join(subclass_directory, "images")
    os.makedirs(os.path.join(merged_folder, "images"), exist_ok=True)
    for image_file in os.listdir(original_images):
        original_image_path = os.path.join(original_images, image_file)
        merged_image_path = os.path.join(merged_folder, "images", image_file)
        shutil.copy(original_image_path, merged_image_path)

    # Copying original labels into merged folder
    original_labels = os.path.join(subclass_directory, "labels")
    os.makedirs(os.path.join(merged_folder, "labels"), exist_ok=True)
    for label_file in os.listdir(original_labels):
        original_label_path = os.path.join(original_labels, label_file)
        merged_label_path = os.path.join(merged_folder, "labels", label_file)
        shutil.copy(original_label_path, merged_label_path)

    # Create "images" directory in merged folder
    os.makedirs(os.path.join(merged_folder, "images"), exist_ok=True)

    # Copying augmented images into merged folder
    if os.path.exists(output_folder):
        augmented_images, augmented_labels = augmented_and_labels  # Capture the returned values

        augmented_images_folder = os.path.join(output_folder, "images")
        for image_file in os.listdir(augmented_images_folder):
            image_path = os.path.join(augmented_images_folder, image_file)
            merged_image_path = os.path.join(merged_folder, "images", image_file)

            print(f"Copying from: {image_path}")
            print(f"Copying to: {merged_image_path}")

            if os.path.exists(image_path):
                shutil.copy(image_path, merged_image_path)
            else:
                print(f"Warning: Source file does not exist - {image_path}")

        # Copying augmented labels into merged folder
        augmented_labels_folder = os.path.join(output_folder, "labels")
        for label_file in os.listdir(augmented_labels_folder):
            label_path = os.path.join(augmented_labels_folder, label_file)
            merged_label_path = os.path.join(merged_folder, "labels", label_file)
            shutil.copy(label_path, merged_label_path)


Mounted at /content/drive
Copying from: /content/drive/MyDrive/capstone/Augmented Data/pascal/Day_1/augment/images/20231028_195643_augmented.jpg
Copying to: /content/drive/MyDrive/capstone/Augmented Data/pascal/Day_1/merged_data/images/20231028_195643_augmented.jpg
Copying from: /content/drive/MyDrive/capstone/Augmented Data/pascal/Day_1/augment/images/20231028_195700_augmented.jpg
Copying to: /content/drive/MyDrive/capstone/Augmented Data/pascal/Day_1/merged_data/images/20231028_195700_augmented.jpg
Copying from: /content/drive/MyDrive/capstone/Augmented Data/pascal/Day_1/augment/images/20231028_195712_augmented.jpg
Copying to: /content/drive/MyDrive/capstone/Augmented Data/pascal/Day_1/merged_data/images/20231028_195712_augmented.jpg
Copying from: /content/drive/MyDrive/capstone/Augmented Data/pascal/Day_1/augment/images/20231028_195723_augmented.jpg
Copying to: /content/drive/MyDrive/capstone/Augmented Data/pascal/Day_1/merged_data/images/20231028_195723_augmented.jpg
Copying from: 